In [1]:
import sys, os, copy, time
sys.path.append('/home/achint/Generative_Modelling/Ziyi_3/POISEVAE-main')
import random
import numpy as np
import torch
HOME_PATH = os.path.expanduser('~')

import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import matplotlib.pyplot as plt
import poisevae
from poisevae.datasets import MNIST_SVHN
from poisevae.networks.PixelCNN_SVHN_MNIST import EncMNIST, DecMNIST, EncSVHN, DecSVHN
from poisevae.networks.pixelcnn import PixelCNN

In [2]:
MNIST_PATH = "/home/achint/datasets/MNIST/%s.pt"
SVHN_PATH = "/home/achint/datasets/SVHN/%s_32x32.mat"

joint_dataset_train = MNIST_SVHN(mnist_pt_path=MNIST_PATH % 'training',svhn_mat_path=SVHN_PATH % 'train')
joint_dataset_test = MNIST_SVHN(mnist_pt_path=MNIST_PATH % 'test',svhn_mat_path=SVHN_PATH % 'test')

In [3]:
batch_size = 128
train_loader = torch.utils.data.DataLoader(joint_dataset_train, batch_size=batch_size, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(joint_dataset_test, batch_size=batch_size, shuffle=True, drop_last=True)
len(train_loader), len(test_loader)

(468, 78)

In [6]:
lat1, lat2 = 20, 20
color_level = 256
device = 'cuda' if torch.cuda.is_available() else 'cpu'

enc_mnist = EncMNIST(lat1).to(device)
dec_mnist = DecMNIST(PixelCNN(lat1, 1, color_level), color_level).to(device)
enc_svhn  = EncSVHN(lat2).to(device)
dec_svhn  = DecSVHN(PixelCNN(lat2, 1, color_level), color_level).to(device)

# Options: 'derivative_autograd', 'derivative_gradient', and 'std_normal'
vae = poisevae.POISEVAE([enc_mnist, dec_mnist], [enc_svhn, dec_svhn], latent_dims=[lat1, (lat2,1,1)], 
                        enc_config='nu', KL_calc='derivative_autograd', batch_size=batch_size
                       ).to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)
writer = None

In [7]:
epoch = 0
epochs = 1 + epoch
for epoch in tqdm(range(epoch, epochs)):
    poisevae.utils.train(vae, train_loader, optimizer, epoch, kl_weight=1, n_gibbs_iter=5, writer=writer, 
                         dec_kwargs={'generate_mode':False})

  0%|                                                     | 0/1 [00:00<?, ?it/s]


TypeError: forward() missing 2 required positional arguments: 'x' and 'generate_mode'